In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2   

In [1]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [9]:
engine = create_engine("sqlite:///final_project")

In [4]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [64]:
results = client.get("xwdj-i9he", limit=1160000)

In [65]:
results_df = pd.DataFrame.from_records(results)

In [166]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_map_tile,sr_location_zip_code,Created Date,Closed Date,Request Year,Open Time,Open Time delta,Total Seconds,Response Time,Department
563842,"4007 COMMERCIAL CENTER DR, AUSTIN, TX 78744",2,Phone,2018-07-25T09:32:18.000,3115602.74743828,Closed,Road Markings/Striping - New,TRAVIS,645J,-97.74023181,...,MJ17,78744,2018-06-06 13:14:59,2018-07-25 09:32:18,2018,48 days 20:17:19,48 days 20:17:19,4220239.0,Slow Response,Austin Transportation
482590,"11419 BARBROOK CV, AUSTIN, TX 78726",6,Phone,2018-12-03T11:53:19.000,3085106.407773,Closed,Street Light Issue- Address,TRAVIS,463A,-97.83095666,...,ME36,78726,2018-12-03 10:37:58,2018-12-03 11:53:19,2018,0 days 01:15:21,0 days 01:15:21,4521.0,Quick Response,Public Works
404768,"119 E 10TH ST, AUSTIN, TX 78701",9,Phone,2019-07-16T15:57:15.000,3115091.3932557,Closed,Injured / Sick Animal,TRAVIS,585S,-97.74025031,...,MJ22,78701,2019-05-23 19:12:08,2019-07-16 15:57:15,2019,53 days 20:45:07,53 days 20:45:07,4653907.0,Slow Response,Animal Services
326920,"E RIVERSIDE DR & GROVE BLVD, AUSTIN, TX",3,Phone,2019-10-15T19:54:53.000,3125672.06830341,Closed,Traffic Signal - Maintenance,TRAVIS,616W,-97.70793115,...,ML19,78741,2019-10-15 19:13:48,2019-10-15 19:54:53,2019,0 days 00:41:05,0 days 00:41:05,2465.0,Quick Response,Public Works
439880,"7200 EASY WIND DR, AUSTIN, TX 78757",7,Phone,2019-03-20T12:47:26.000,3121068.23827035,Closed,Dockless Mobility,TRAVIS,525U,-97.71944109,...,MK28,78757,2019-03-20 06:12:41,2019-03-20 12:47:26,2019,0 days 06:34:45,0 days 06:34:45,23685.0,Quick Response,Austin Transportation
351912,"328 E 6TH ST, AUSTIN, TX 78701",9,Spot311 Interface,2019-09-06T09:08:58.000,3115367.27884333,Closed,Shared Micromobility,TRAVIS,585X,-97.73948899,...,MJ22,78701,2019-08-28 12:00:57,2019-09-06 09:08:58,2019,8 days 21:08:01,8 days 21:08:01,767281.0,Slow Response,Austin Transportation
583420,"LEDESMA RD & CESSAL AVE, AUSTIN, TX 78721",3,Phone,2018-05-16T07:21:19.000,3132862.74813981,Closed,School Zone Flasher - Timing/Maintenance,TRAVIS,586U,-97.68394433,...,MM22,78721,2018-04-25 14:36:49,2018-05-16 07:21:19,2018,20 days 16:44:30,20 days 16:44:30,1788270.0,Slow Response,Austin Transportation
454776,"4600 BUNDYHILL DR, AUSTIN, TX 78723",1,Phone,2019-08-21T01:03:26.000,3134722.24815463,Closed,Creek & Pond Vegetation Control,TRAVIS,586G,-97.6776283,...,MM24,78723,2019-02-13 16:20:32,2019-08-21 01:03:26,2019,188 days 08:42:54,188 days 08:42:54,16274574.0,Slow Response,Watershed Check
422819,"9503 STONEBRIDGE DR, AUSTIN, TX 78758",4,Web,2019-04-23T08:49:29.000,3125618.49840046,Closed,Austin Code - Request Code Officer,TRAVIS,496W,-97.70423212,...,ML31,78758,2019-04-23 08:39:31,2019-04-23 08:49:29,2019,0 days 00:09:58,0 days 00:09:58,598.0,Quick Response,Austin Code
538180,"4410 JINX AVE, AUSTIN, TX 78745",5,Spot311 Interface,2018-07-31T20:48:21.000,3103182.75116488,Closed,Austin Code - Request Code Officer,TRAVIS,644B,-97.77924516,...,MG18,78745,2018-07-31 20:39:16,2018-07-31 20:48:21,2018,0 days 00:09:05,0 days 00:09:05,545.0,Quick Response,Austin Code


In [122]:
len(results_df)

329220

In [67]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [69]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [74]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [77]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [78]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [161]:
median = results_df['Total Seconds'].median(axis=0)

In [162]:
median

69911.0

In [163]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [164]:
print(results_df['Total Seconds'].max())

31495482.000000004


In [145]:
results_df.loc[results_df['Total Seconds'] == 109367883]

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_map_tile,sr_location_zip_code,Created Date,Closed Date,Request Year,Open Time,Open Time delta,Total Seconds,Response Time,Department
609039,"1520 RUTHERFORD LN, AUSTIN, TX 78754",1,Phone,2021-08-05T14:23:43.000,3133220.45539295,Closed,Animal Control - Assistance Request,TRAVIS,526Y,-97.68097996,...,MM28,78754,2018-02-16 18:25:40,2021-08-05 14:23:43,2018,1265 days 19:58:03,1265 days 19:58:03,109367883.0,Slow Response,Animal Services


In [159]:
np.nanpercentile(results_df['Total Seconds'], 99.35) 

31897949.610000003

In [160]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [165]:
response = results_df['Response Time'].value_counts()
response

Quick Response    163164
Slow Response     163163
Name: Response Time, dtype: int64

In [82]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [83]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [84]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [89]:
top_dept = results_df['Department'].value_counts()
top_dept

Austin Code                 75903
Animal Services             61435
Austin Transportation       57107
Public Works                56829
Austin Resource Recovery    51077
Parks and Recreation         8755
Public Health                8043
Watershed Check              4932
Austin Water                 4375
Housing and Planning          140
Austin Fire                    73
Name: Department, dtype: int64

In [98]:
print(results_df.columns)

Index(['sr_location', 'sr_location_council_district',
       'sr_method_received_desc', 'sr_closed_date', 'sr_location_x',
       'sr_status_desc', 'sr_type_desc', 'sr_location_county',
       'sr_location_map_page', 'sr_location_long', 'sr_updated_date',
       'sr_number', 'sr_location_lat', 'sr_location_lat_long',
       'sr_status_date', 'sr_location_street_name', 'sr_created_date',
       'sr_location_street_number', 'sr_location_city', 'sr_location_y',
       'sr_location_map_tile', 'sr_location_zip_code', 'Created Date',
       'Closed Date', 'Request Year', 'Open Time', 'Open Time delta',
       'Total Seconds', 'Response Time', 'Department'],
      dtype='object')


In [95]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [167]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [168]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [108]:
training_df = training_df.drop(columns=[])

In [170]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [171]:
training_df.head(10)

,Report Method,Zip Code,Request Year,Total Seconds,Response Time,Department
294161,Phone,78723,2019,152190.0,Slow Response,Austin Code
294162,Spot311 Interface,78744,2019,152934.0,Slow Response,Austin Code
294163,Web,78701,2019,113327.0,Slow Response,Public Works
294164,Spot311 Interface,78701,2019,121696.0,Slow Response,Austin Resource Recovery
294165,Phone,78701,2019,155392.0,Slow Response,Austin Code
294166,Phone,78745,2019,18479378.0,Slow Response,Animal Services
294167,Spot311 Interface,78745,2019,155819.0,Slow Response,Austin Code
294168,Phone,78746,2019,1097446.0,Slow Response,Austin Resource Recovery
294169,Spot311 Interface,78745,2019,117788.0,Slow Response,Animal Services
294170,Spot311 Interface,78660,2019,162582.0,Slow Response,Animal Services


In [ ]:
connection = engine.raw_connection()
cursor = connection.cursor()

#df is the dataframe containing an index and the columns "Event" and "Day"
#create Index column to use as primary key
df.reset_index(inplace=True)
df.rename(columns={'index':'Index'}, inplace =True)

#create the table but first drop if it already exists
command = '''DROP TABLE IF EXISTS localytics_app2;
CREATE TABLE localytics_app2
(
"Index" serial primary key,
"Event" text,
"Day" timestamp without time zone,
);'''
cursor.execute(command)
connection.commit()

#stream the data using 'to_csv' and StringIO(); then use sql's 'copy_from' function
output = cStringIO.StringIO()
#ignore the index
df.to_csv(output, sep='\t', header=False, index=False)
#jump to start of stream
output.seek(0)
contents = output.getvalue()
cur = connection.cursor()
#null values become ''
cur.copy_from(output, 'localytics_app2', null="")    
connection.commit()
cur.close()